# Desirability Functions

Desirability functions are mathematical constructs used to quantify the preferability of different outcomes in multi-objective optimization problems. These functions depend on multiple variables, which can be categorized into two main types:

1. **Parameters**: Also referred to as coefficient parameters or shape parameters, these define the functional form of the desirability function. Each desirability function is characterized by a unique set of shape parameters, denoted as $\theta = (\theta_1, \theta_2, ..., \theta_n)$, where each $\theta_i$ has its specific type and valid range.

2. **Input Variables**: These are the variables to which the desirability function is applied, represented as $x = (x_1, x_2, ..., x_m)$.

The general form of a desirability function can be expressed as:
  
$D(x; \theta) : \mathbb{R}^m \times \Theta \rightarrow [0, 1]$

where $\Theta$ represents the parameter space. The output of a desirability function is typically normalized to the range [0, 1], where 0 represents the least desirable outcome and 1 represents the most desirable outcome.

## API Description

PUMAS includes a family of desirability functions implemented as a set of parameterized strategies. The implementation requires two steps to use a desirability function:

1. **Initialization**: The strategy is initialized with the shape parameters $\theta$.

   ```python
   desirability_instance = desirability_class(params=shape_parameters)
   ```

2. **Computation**: The desirability function is computed for a given input $x$.

   ```python
   y = desirability_instance(x=x_input)
   ```

Once initialized, the desirability function can be applied to multiple inputs while maintaining its overall functional shape.

## Implementation Details

This implementation strikes a balance between flexibility and standardization, facilitating easy extension of the desirability function family with new members that may require different parameters but operate on the same input space.

While the `compute` method remains consistent across all members of the family, the parameters vary for each function. This variability presents a known challenge in the parameterized strategy pattern, as it necessitates that the user or software interacts with a variable API. Traditionally, this issue is addressed by coupling a data model for parameters to the respective strategy.

Our implementation, however, offers an innovative approach based on self-discovery and validation. Each desirability function in our framework:

1. Exposes a detailed description of its required parameters
2. Embeds reasonable default values where possible
3. Defines acceptable ranges for parameters when appropriate

Furthermore, the input parameter values are validated against this internal definition, raising descriptive error messages when requirements are not met. This approach can be formalized as follows:

Let $F$ be the set of all desirability functions in our family. For each function $D \in F$, we define:

$\text{parameters}(D) = \{\theta_1, \theta_2, ..., \theta_n\}$

Where each specific parameter $\theta_i$ is defined by a tuple. In the case of numeric *float* parameters, this tuple is:

$\theta_i = (\text{name}, \text{type}, \text{default}, \text{range})$

The validation process for a given input parameter set $\theta$ can be represented as:

$\text{validate}(D, \theta) = \begin{cases} 
      \text{True } & \text{if } \forall \theta_i \in \text{parameters}(D), \theta_i \in \text{range}(\theta_i) \\
      \text{False } & \text{otherwise}
   \end{cases}$

Where $\text{range}(\theta_i)$ represents the valid range for parameter $\theta_i$.  

The parameter definition and validation procedure differ for other types of parameters, such as *integer*, *boolean*, *str*, *iterable*, and *mapping*.

This self-discovery and validation mechanism enhances the robustness and usability of our desirability function framework, allowing for more intuitive and error-resistant implementation in various optimization scenarios.


## Code Examples

In [15]:
# import the library and print the version
import pumas
print(pumas.__version__)

0.0.0


### Discover available desirability functions
A catalogue contains all the desirability functions implemented in PUMAS.   
The catalogue can be extended by registering into it new classes that adhere to the Desirability interface. 

In [16]:
from pumas.desirability import desirability_catalogue
desirability_catalogue.list_items()

['sigmoid',
 'double_sigmoid',
 'bell',
 'sigmoid_bell',
 'multistep',
 'leftstep',
 'rightstep',
 'step',
 'value_mapping']

The catalogue yields a desirability function class, not an instance.

In [17]:
desirability_class = desirability_catalogue.get("sigmoid")

## Initialize and use a desirability function
The instantiation requires a number of parameters.

In [18]:
desirability_instance = desirability_class(params={"low": 0.0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0})

Once properly instantiated, the desirability function can be used to calculate desirability scores.

In [21]:
result = desirability_instance.compute_numeric(x=0.5)
print(f"The desirability score is {result}")

The desirability score is 0.5


The initialized desirability function can be used on multiple input

In [29]:
input_list = [x / 10.0 for x in range(1, 10, 1)]
result_list = [desirability_instance.compute_numeric(x=x) for x in input_list]
for x, y in zip(input_list,result_list):
    print(f"D({x:.2f}) = {y:.2f}")

D(0.10) = 0.28
D(0.20) = 0.33
D(0.30) = 0.39
D(0.40) = 0.44
D(0.50) = 0.50
D(0.60) = 0.56
D(0.70) = 0.61
D(0.80) = 0.67
D(0.90) = 0.72


### Discover Parameters
If the nature of the parameters for a given desirability function is not known it is possible to discover them.  
It is possible to instantiate a desirability function without any parameters.   
This object is not ready to compute a score, but it offers a detailed description of the required parameters.

In [32]:
desirability_class = desirability_catalogue.get("sigmoid")
desirability_instance_blank = desirability_class()

Depending on the individual desirability function some parameters default to a reasonable value. 
Other parameters default to None: it is mandatory to set them with an appropriate value.  

The parameters_map attribute contains a detailed overview of the parameters, including their type, and attributes.
This map can guide the setting of appropriate parameter values.

In [35]:
desirability_instance_blank.parameters_map

{'low': FloatParameter(name='low', default=None, min=-inf, max=inf),
 'high': FloatParameter(name='high', default=None, min=-inf, max=inf),
 'k': FloatParameter(name='k', default=0.5, min=-1.0, max=1.0),
 'base': FloatParameter(name='base', default=10.0, min=1.0, max=10.0),
 'shift': FloatParameter(name='shift', default=0.0, min=0.0, max=1.0)}

An overview of the current value of each parameter is available. 

In [36]:
print(desirability_instance_blank.get_parameters_values())

{'low': None, 'high': None, 'k': 0.5, 'base': 10.0, 'shift': 0.0}


Once the type and attributes of parameters is known, it is possible to set the desired parameter values directly on the blank instance as an alternative to instantiate another object. This represents a slightly faster alternative to instantiating a new object.  
In either case the current state of the parameters reflect the input. 


In [38]:
desirability_instance_blank.set_parameters_values(
    {"low": 0.0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0}
)
print(desirability_instance_blank.get_parameters_values())

{'low': 0.0, 'high': 1.0, 'k': 0.1, 'base': 10.0, 'shift': 0.0}


In [39]:
x= 0.5
y = desirability_instance_blank(x=x)
print(f"D({x:.2f}) = {y:.2f}")

D(0.50) = 0.50


### Errors while setting parameters

In [42]:
# missing a mandatory parameter raises an error while executing the computation
desirability_instance = desirability_class(params={ "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0})
try:
    desirability_instance(x=0.5)
except Exception as e:
    print(e)

All parameters must be set (non-None) before computation. Please set the value of ['low']


In [45]:
# providing a wrong type raises an error during initialization
try:
    desirability_instance = desirability_class(params={ "low": 0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0})
except Exception as e:
    print(e)

Error in parameter 'low': Expected type float, got int instead.


In [47]:
# providing a value outside the range raises an error during initialization
try:
    desirability_instance = desirability_class(params={ "low": 0.0, "high": 1.0, "k": 0.1, "shift": 10.0, "base": 10.0})
except Exception as e:
    print(e)

Error in parameter 'shift': Parameter Value 10.0 is outside the allowed range [0.0, 1.0].


### Errors while providing input

In [49]:
# providing the wrong type: str instead of float
desirability_instance = desirability_class(params={ "low": 0.0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0})
try:
    desirability_instance(x="0.5")
except Exception as e:
    print(e)

Expected float, got str instead.


In [50]:
# providing the wrong type:  int instead of float
desirability_instance = desirability_class(params={ "low": 0.0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0})
try:
    desirability_instance(x=5)
except Exception as e:
    print(e)

Expected float, got int instead.


In [51]:
# providing the wrong type: list instead of float
desirability_instance = desirability_class(params={ "low": 0.0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0})
try:
    desirability_instance(x=[0.5])
except Exception as e:
    print(e)

Expected float, got list instead.
